# Analyse von Regierungsdaten

---

- Was sind die 5 häufigsten Betriebssysteme, mit denen auf https://usa.gov zugegriffen wird?

---

Datenquelle: https://analytics.usa.gov/data -> `Operating systems`

LLM-Chat: https://chatgpt.com/share/6932a30d-b100-8009-ab85-f168a43a65f3

## Daten einlesen

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://analytics.usa.gov/data/live/os-90-days.csv")

Rohdaten lokal sichern - sicher ist sicher!

In [ ]:
#df.to_csv("os-90-days.csv")

Falls die Online-Datenquelle einmal nicht verfügbar ist, lesen wir unsere Sicherungskopie ein

In [ ]:
#df = pd.read_csv("os-90-days.csv")

Ein erster Blick auf die Daten

In [ ]:
df

## Daten vorbereiten

Wir Teilen die Spalte 'Month-Year' in zwei Spalten

In [ ]:
df[['Month', 'Year']] = df['Month-Year'].str.split(' ', expand=True)

In [ ]:
df

## Daten analysieren

Einen ersten Überblick verschaffen

In [ ]:
df.groupby(["Year", "os"]).describe()

Um sinnvollen Aussagen treffen zu können, müssen wir die OS-Namen auf einige wenige Kategorien zurückführen.

In [ ]:
import numpy as np

def map_os_category(os_name):
    os_name = str(os_name).lower()

    if any(x in os_name for x in ["win32", "win64", "windows", "windows phone", "xbox"]):
        return "Windows"

    elif any(x in os_name for x in ["mac os", "macos", "macintosh", "macintel", "ios"]):
        return "MacOS"

    elif any(x in os_name for x in ["linux", "freebsd", "openbsd", "netbsd", "sunos", "unix", "android", "samsung"]):
        return "Linux"

    elif any(x in os_name for x in ["chrome", "ChromeOS"]):
        return "ChromeOS"

    elif any(x in os_name for x in [
        "xbox", "playstation", "nintendo", "ps3", "ps4", "ps5", "wii"
    ]):
        return "GameConsole"

    else:
        return os_name
        #return "Other"

In [ ]:
df["os_category"] = df["os"].apply(map_os_category)

In [ ]:
df.groupby(["os_category"]).describe()

In [ ]:
df.groupby(["os_category"])["Active Users"].sum().sort_values(ascending=False)

Die ersten `n` Betriebssysteme als Balkendiagramm

In [ ]:
n = 7

In [ ]:
df.groupby(["os_category"])["Active Users"].sum().sort_values(ascending=False).head(n).plot.bar()

und noch in Prozent statt absoluten Zahlen

In [ ]:
df_os_pct = (
    df.groupby("os_category", as_index=False)["Active Users"]
      .sum()
      .assign(Percentage=lambda x: 100 * x["Active Users"] / x["Active Users"].sum())
      .sort_values("Active Users", ascending=False)
)

In [ ]:
df_os_pct

In [ ]:
df_os_pct.set_index("os_category")[["Percentage"]].head(n).plot.bar()